In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds

In [2]:
(train_dataset, val_dataset), metadata = tfds.load('tf_flowers', 
                                                     split=tfds.Split.TRAIN.subsplit([70, 30]), 
                                                     as_supervised=True, with_info=True) 
classes = np.array(metadata.features['label'].names)

In [3]:
# Size of training and test sets
train_size = 0
val_size = 0
for example in train_dataset:
    train_size += 1
for example in val_dataset:
    val_size += 1
print("Number of training examples: {}".format(train_size))
print("Number of training examples: {}".format(val_size))

IMG_SIZE = 224
# Normalise data
def formatting(images, labels):
    images = tf.cast(images, tf.float32)
    images = tf.image.resize(images, (IMG_SIZE, IMG_SIZE))/255
    return images, labels

train_dataset = train_dataset.map(formatting)
val_dataset = val_dataset.map(formatting)

Number of training examples: 2590
Number of training examples: 1080


In [4]:
# Define model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', data_format='channels_last'),
    tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', data_format='channels_last'),
    tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    #tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', data_format='channels_last'),
    #tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])
# Compile model
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer = 'adam',
              metrics=['accuracy'])
#model.summary()

In [5]:
# Shuffling and splitting of training set 
BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(train_size).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [6]:
# Fit model
history = model.fit(train_dataset,
                   epochs= 5,
                   steps_per_epoch=np.ceil(train_size/BATCH_SIZE),
                   validation_data=val_dataset,)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


81/81 [==============================] - 318s 4s/step - loss: 1.8511 - accuracy: 0.3672 - val_loss: 1.1634 - val_accuracy: 0.5176
Epoch 2/5
81/81 [==============================] - 348s 4s/step - loss: 1.0213 - accuracy: 0.5731 - val_loss: 1.1144 - val_accuracy: 0.5861
Epoch 3/5
81/81 [==============================] - 344s 4s/step - loss: 0.7433 - accuracy: 0.6973 - val_loss: 1.0300 - val_accuracy: 0.6361
Epoch 4/5
81/81 [==============================] - 354s 4s/step - loss: 0.4793 - accuracy: 0.8188 - val_loss: 1.3441 - val_accuracy: 0.5824
Epoch 5/5
81/81 [==============================] - 335s 4s/step - loss: 0.3144 - accuracy: 0.8859 - val_loss: 1.2836 - val_accuracy: 0.5963
